In [12]:
import simexpal
import networkit as nk
from networkit import vizbridges
import pandas as pd
import re
import osmnx
import networkx as nx
import folium
from ipywidgets import Box
from collections import defaultdict
import numpy as np
import itertools
import momepy
import io
from PIL import Image

import visfunctions
import solveExactForestIndex
import solveExactHarmonicResistance

pd.options.display.width = 0
pd.set_option('max_colwidth', 800)

# Load results

In [13]:
# load simex results

def parse(run, f):
    res = {}
    res['status'] = run.get_status().name
    res['experiment'] = run.experiment.name
    res['instance'] = run.instance.shortname
    res['run'] = run
    for var in run.experiment.variation:
        res[var.axis] = var.name
    
    if run.experiment.name == 'exactSolution':
        for line in f.readlines():
            if line.startswith('time:'):
                res['time'] = line[7:-1]
            if line.startswith('best edges:'):
                edges = re.findall(r'\(\s?(\d+),\s?(\d+)\)', line)
                res['edges'] = [(int(e[0]), int(e[1])) for e in edges]
    else:
        for line in f.readlines():
            if line.startswith('Running time:'):
                res['time'] = line[14:-1]
            if line.startswith('Result edges:'):
                edges = re.findall(r'\(\s?(\d+),\s?(\d+)\)', line)
                res['edges'] = [(int(e[0]), int(e[1])) for e in edges]
    return res

# load all results
cfg = simexpal.config_for_dir('~/robustness/harmonic-resistance-case-study')
results = []
for run in cfg.discover_all_runs():
    if run.get_status() == simexpal.base.Status.NOT_SUBMITTED: continue
    try:
        with run.open_output_file() as f:
            results.append(parse(run, f))
    except RuntimeError as e:
        pass
        # print('could not open file for run', e, 'run status:', run.get_status())

results = pd.DataFrame(results)
pd.reset_option('display.max_rows')

# dataframe types
results['status'] = results['status'].astype('category')
results['experiment'] = results['experiment'].astype('category')
results['instance'] = results['instance'].astype('category')
results['time'] = pd.to_timedelta(results['time'])

# filter for THR
resultsTHR = results.query('experiment == "stGreedy" and instance.str.contains("Berlin")', engine='python')
resultsTHR

,status,experiment,instance,run,edges,time
10,STARTED,stGreedy,Berlin-Germany,<simexpal.base.Run object at 0x7f3183499f40>,NaN,NaT
11,FINISHED,stGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(887, 737), (293, 290), (427, 265), (799, 50), (908, 799), (25, 22), (34, 33), (902, 591), (377, 375), (422, 421), (794, 266), (733, 528), (715, 691), (820, 607), (646, 472), (864, 422), (296, 285), (848, 732), (109, 108), (499, 65)]",0 days 00:20:47.882000
12,FINISHED,stGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(637, 633), (685, 682), (152, 150), (3004, 152), (167, 162), (2424, 899), (3370, 2892), (2741, 1622), (162, 51), (3370, 3302), (2918, 2905), (828, 824), (650, 648), (3122, 3061), (2918, 2715), (2344, 2343), (2343, 1145), (647, 645), (2077, 645), (1303, 1301)]",0 days 20:47:30.448000


In [14]:
# load forest index results (different simex setup/directory)
def parse(run, f):
    res = {}
    res['status'] = run.get_status().name
    res['experiment'] = run.experiment.name
    res['instance'] = run.instance.shortname
    res['run'] = run
    for var in run.experiment.variation:
        res[var.axis] = var.name

    else:
        for line in f.readlines():
            if line.startswith('items:'):
                edges = re.findall(r'\(\s?(\d+),\s?(\d+)\)', line)
                res['edges'] = [(int(e[0]), int(e[1])) for e in edges]
    return res

# load all results
cfg = simexpal.config_for_dir('')
resultsFI = []
for run in cfg.discover_all_runs():
    if run.get_status() == simexpal.base.Status.NOT_SUBMITTED: continue
    try:
        with run.open_output_file() as f:
            resultsFI.append(parse(run, f))
    except RuntimeError as e:
        pass
        # print('could not open file for run', e, 'run status:', run.get_status())

resultsFI = pd.DataFrame(resultsFI).query('experiment == "forestIndexGreedy" and instance.str.contains("Berlin")', engine='python')
# dataframe types
resultsFI['status'] = resultsFI['status'].astype('category')
resultsFI['experiment'] = resultsFI['experiment'].astype('category')
resultsFI['instance'] = resultsFI['instance'].astype('category')

resultsFI

,status,experiment,instance,run,k,edges
11,FINISHED,forestIndexGreedy,Berlin-Germany,<simexpal.base.Run object at 0x7f3183490a90>,k-20,"[(26283, 26282), (17669, 17668), (28654, 22053), (31268, 31267), (28151, 28150), (28886, 28885), (17916, 17914), (32558, 28088), (28292, 28291), (32885, 32884), (28699, 28698), (26451, 26450), (27008, 27007), (31237, 31236), (29230, 29229), (29696, 29695), (23567, 23566), (33204, 33203), (26880, 26879), (30967, 30966)]"
12,FINISHED,forestIndexGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490af0>,k-20,"[(664, 663), (986, 977), (666, 665), (701, 699), (941, 844), (844, 843), (304, 303), (305, 303), (753, 506), (506, 505), (725, 724), (776, 724), (891, 846), (846, 845), (808, 672), (672, 671), (964, 939), (939, 938), (718, 717), (717, 716)]"
13,FINISHED,forestIndexGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490b50>,k-20,"[(2618, 1269), (816, 814), (2629, 2628), (3180, 3179), (3365, 3364), (2625, 2624), (3589, 2527), (784, 783), (2835, 1229), (3599, 2482), (2406, 2404), (2864, 2863), (2288, 2287), (1481, 1480), (2142, 2141), (2837, 2836), (569, 100), (3334, 566), (884, 882), (3449, 3448)]"


In [15]:
manualChoice = [
    {
        'experiment': 'allBridges',
        'instance': 'Mitte-Berlin-Germany',
        'run': results.query('instance == "Mitte-Berlin-Germany"').iloc[0].run,
        'OSMIds': [
            # Marschallbrücke
            (26861434, 26747882),

            # Weidenhammer Brücke
            (10774556436, 1769589042),

            # Ebertsbrücke
            (27412113, 27412072),

            # Liebknechtbrücke
            (29221504, 1472911017),
            (697323164, 29221506),

            # Rathausbrücke
            (262479941, 29219469),

            # Mühlendammbrücke
            (10567768816, 21487285),
            (29207836, 29207861),

            # Jannowitzbrücke
            (21487224, 29215073),
            (660778774, 196725581),
        ],
    },
{
        'experiment': 'allBridges',
        'instance': 'Treptow-Köpenick-Berlin-Germany',
        'run': results.query('instance == "Treptow-Köpenick-Berlin-Germany"').iloc[0].run,
        'OSMIds': [
            # - Minna-Todenhagen-Brücke
            (11456395826, 8354024836),
            (11456395827, 8244197150),
            (8354024891, 11456395828),
            (8244197157, 9887626934),

            # - Stubenrauchbrücke
            (244149835, 21432653),
            (8093961555, 529132504),

            # - Treskowbrücke
            (3186398219, 276248266),

            # - falsche Kante über die Spree
            (1562864534, 505810033),

            # - Wilhelm-Spindler-Brücke
            (9886655276, 297100517),
            (257818028, 9886655277),

            # - Lange Brücke
            (1576630672, 1576655246),
            (8662312723, 3522290030),
        ],
    }
]

def osmIdsToEdgeIds(row: pd.Series):
    g, sourceIds, destIds, _ = visfunctions.loadGraph(row.run)
    edges = []
    for u, v in g.iterEdges():
        edgeId = g.edgeId(u,v)
        for osm_u, osm_v in row.OSMIds:
            if str(osm_u) in (sourceIds[edgeId], destIds[edgeId]) and str(osm_v) in (sourceIds[edgeId], destIds[edgeId]):
                edges.append((u,v))
    return edges


manualChoice = pd.DataFrame(manualChoice)
manualChoice['edges'] = manualChoice.apply(osmIdsToEdgeIds, axis=1)

manualChoice

,experiment,instance,run,OSMIds,edges
0,allBridges,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(26861434, 26747882), (10774556436, 1769589042), (27412113, 27412072), (29221504, 1472911017), (697323164, 29221506), (262479941, 29219469), (10567768816, 21487285), (29207836, 29207861), (21487224, 29215073), (660778774, 196725581)]","[(62, 508), (65, 499), (68, 70), (69, 569), (91, 788), (92, 524), (265, 427), (266, 794), (290, 293), (514, 606)]"
1,allBridges,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(11456395826, 8354024836), (11456395827, 8244197150), (8354024891, 11456395828), (8244197157, 9887626934), (244149835, 21432653), (8093961555, 529132504), (3186398219, 276248266), (1562864534, 505810033), (9886655276, 297100517), (257818028, 9886655277), (1576630672, 1576655246), (8662312723, 3522290030)]","[(33, 37), (39, 2698), (441, 445), (842, 2540), (850, 2689), (1340, 2955), (2203, 2204), (2699, 3431), (2892, 3370), (3360, 3459), (3430, 3433), (3432, 3626)]"


In [16]:
resultsTHR = resultsTHR.assign(experiment='harmonicResistanceGreedy') 
results = pd.concat((resultsTHR, resultsFI, manualChoice)).reset_index()[['experiment', 'instance', 'run', 'edges']]
results

,experiment,instance,run,edges
0,harmonicResistanceGreedy,Berlin-Germany,<simexpal.base.Run object at 0x7f3183499f40>,NaN
1,harmonicResistanceGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(887, 737), (293, 290), (427, 265), (799, 50), (908, 799), (25, 22), (34, 33), (902, 591), (377, 375), (422, 421), (794, 266), (733, 528), (715, 691), (820, 607), (646, 472), (864, 422), (296, 285), (848, 732), (109, 108), (499, 65)]"
2,harmonicResistanceGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(637, 633), (685, 682), (152, 150), (3004, 152), (167, 162), (2424, 899), (3370, 2892), (2741, 1622), (162, 51), (3370, 3302), (2918, 2905), (828, 824), (650, 648), (3122, 3061), (2918, 2715), (2344, 2343), (2343, 1145), (647, 645), (2077, 645), (1303, 1301)]"
3,forestIndexGreedy,Berlin-Germany,<simexpal.base.Run object at 0x7f3183490a90>,"[(26283, 26282), (17669, 17668), (28654, 22053), (31268, 31267), (28151, 28150), (28886, 28885), (17916, 17914), (32558, 28088), (28292, 28291), (32885, 32884), (28699, 28698), (26451, 26450), (27008, 27007), (31237, 31236), (29230, 29229), (29696, 29695), (23567, 23566), (33204, 33203), (26880, 26879), (30967, 30966)]"
4,forestIndexGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490af0>,"[(664, 663), (986, 977), (666, 665), (701, 699), (941, 844), (844, 843), (304, 303), (305, 303), (753, 506), (506, 505), (725, 724), (776, 724), (891, 846), (846, 845), (808, 672), (672, 671), (964, 939), (939, 938), (718, 717), (717, 716)]"
5,forestIndexGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490b50>,"[(2618, 1269), (816, 814), (2629, 2628), (3180, 3179), (3365, 3364), (2625, 2624), (3589, 2527), (784, 783), (2835, 1229), (3599, 2482), (2406, 2404), (2864, 2863), (2288, 2287), (1481, 1480), (2142, 2141), (2837, 2836), (569, 100), (3334, 566), (884, 882), (3449, 3448)]"
6,allBridges,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(62, 508), (65, 499), (68, 70), (69, 569), (91, 788), (92, 524), (265, 427), (266, 794), (290, 293), (514, 606)]"
7,allBridges,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(33, 37), (39, 2698), (441, 445), (842, 2540), (850, 2689), (1340, 2955), (2203, 2204), (2699, 3431), (2892, 3370), (3360, 3459), (3430, 3433), (3432, 3626)]"


# Berlin case study: analytical evaluation

In [17]:
results = results.query('instance == "Mitte-Berlin-Germany" or instance == "Treptow-Köpenick-Berlin-Germany"')
results

,experiment,instance,run,edges
1,harmonicResistanceGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(887, 737), (293, 290), (427, 265), (799, 50), (908, 799), (25, 22), (34, 33), (902, 591), (377, 375), (422, 421), (794, 266), (733, 528), (715, 691), (820, 607), (646, 472), (864, 422), (296, 285), (848, 732), (109, 108), (499, 65)]"
2,harmonicResistanceGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(637, 633), (685, 682), (152, 150), (3004, 152), (167, 162), (2424, 899), (3370, 2892), (2741, 1622), (162, 51), (3370, 3302), (2918, 2905), (828, 824), (650, 648), (3122, 3061), (2918, 2715), (2344, 2343), (2343, 1145), (647, 645), (2077, 645), (1303, 1301)]"
4,forestIndexGreedy,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490af0>,"[(664, 663), (986, 977), (666, 665), (701, 699), (941, 844), (844, 843), (304, 303), (305, 303), (753, 506), (506, 505), (725, 724), (776, 724), (891, 846), (846, 845), (808, 672), (672, 671), (964, 939), (939, 938), (718, 717), (717, 716)]"
5,forestIndexGreedy,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183490b50>,"[(2618, 1269), (816, 814), (2629, 2628), (3180, 3179), (3365, 3364), (2625, 2624), (3589, 2527), (784, 783), (2835, 1229), (3599, 2482), (2406, 2404), (2864, 2863), (2288, 2287), (1481, 1480), (2142, 2141), (2837, 2836), (569, 100), (3334, 566), (884, 882), (3449, 3448)]"
6,allBridges,Mitte-Berlin-Germany,<simexpal.base.Run object at 0x7f3183499fa0>,"[(62, 508), (65, 499), (68, 70), (69, 569), (91, 788), (92, 524), (265, 427), (266, 794), (290, 293), (514, 606)]"
7,allBridges,Treptow-Köpenick-Berlin-Germany,<simexpal.base.Run object at 0x7f3183497040>,"[(33, 37), (39, 2698), (441, 445), (842, 2540), (850, 2689), (1340, 2955), (2203, 2204), (2699, 3431), (2892, 3370), (3360, 3459), (3430, 3433), (3432, 3626)]"


In [18]:
# compute THR and FI score for all instances

def FIGain(row: pd.Series, k=25):
    g, _, _, _ = visfunctions.loadGraph(row.run)
    base = solveExactForestIndex.forestIndex(g)
    for i, edge in enumerate(row.edges):
        if i >= k: break
        g.removeEdge(*edge)
    return abs(base - solveExactForestIndex.forestIndex(g))

def THRGain(row: pd.Series):
    g, _, _, _ = visfunctions.loadGraph(row.run)
    base = solveExactHarmonicResistance.totalHarmonicResistance(g)
    for edge in row.edges:
        g.removeEdge(*edge)
    return abs(base - solveExactHarmonicResistance.totalHarmonicResistance(g))

def centralityScore(row: pd.Series):
    graph = nk.readGraph(row.run.instance.fullpath)
    cc = nk.centrality.Closeness(graph, False, nk.centrality.ClosenessVariant.GENERALIZED)
    cc.run()
    nodePercentiles = {}
    prevBcValue = None
    prevNode = None
    numSeen = -1
    numCurrent = 0
    for node, bcValue in reversed(cc.ranking()):
        if prevBcValue and abs(prevBcValue - bcValue) < 1e-6:
            nodePercentiles[node] = nodePercentiles[prevNode]
            numCurrent = numCurrent + 1
        else:
            numSeen = numSeen + numCurrent + 1
            numCurrent = 0
            nodePercentiles[node] = numSeen/graph.numberOfNodes()
            prevBcValue = bcValue
            prevNode = node

    centralities = []
    for edge in row.edges:
        centralities.append(np.mean((nodePercentiles[edge[0]], nodePercentiles[edge[1]])))

    return np.mean(centralities)

results['THR gain'] = results.apply(THRGain, axis=1)
results['FI gain'] = results.apply(FIGain, axis=1)
results['centrality score'] = results.apply(centralityScore, axis=1)

/tmp/ipykernel_532656/977656216.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['THR gain'] = results.apply(THRGain, axis=1)
/tmp/ipykernel_532656/977656216.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results['FI gain'] = results.apply(FIGain, axis=1)
/tmp/ipykernel_532656/977656216.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pand

In [19]:
FIGain(results.query('instance == "Mitte-Berlin-Germany" and experiment == "forestIndexGreedy"').iloc[0], 12)

6333.300118402694

In [21]:
results[['experiment', 'instance', 'THR gain', 'FI gain', 'centrality score']]

,experiment,instance,THR gain,FI gain,centrality score
1,harmonicResistanceGreedy,Mitte-Berlin-Germany,45535.625279,4636.789313,0.648590
2,harmonicResistanceGreedy,Treptow-Köpenick-Berlin-Germany,399917.073212,18863.781827,0.733131
4,forestIndexGreedy,Mitte-Berlin-Germany,3176.499849,10512.206056,0.334416
5,forestIndexGreedy,Treptow-Köpenick-Berlin-Germany,5744.807661,39218.646062,0.283124
6,allBridges,Mitte-Berlin-Germany,71786.247185,2003.911533,0.754381
7,allBridges,Treptow-Köpenick-Berlin-Germany,414213.155573,9114.504798,0.782486


# Berlin case study: Visual results

## Mitte

### Harmonic Resistance

In [24]:
m = visfunctions.map_results(results.query('instance == "Mitte-Berlin-Germany" and experiment == "harmonicResistanceGreedy"'), False, color="#00FF00", map_location=[52.5195,13.3921], zoom=15)
m

no coords for edge (848, 732) wayid: [346240658]


In [25]:
img_data = m._to_png(10)
Image.open(io.BytesIO(img_data)).save('Berlin-Mitte-THR.png')

In [22]:
m = visfunctions.map_results(results.query('instance == "Mitte-Berlin-Germany" and experiment == "harmonicResistanceGreedy"'), False, color="#00FF00", map_location=[52.5195,13.3921], zoom=15)
visfunctions.gjMitte.add_to(m)
m

no coords for edge (848, 732) wayid: [346240658]


In [23]:
img_data = m._to_png()
Image.open(io.BytesIO(img_data)).save('Berlin-Mitte-Boundary.png')

### Forest Index

In [26]:
m = visfunctions.map_results(results.query('instance == "Mitte-Berlin-Germany" and experiment == "forestIndexGreedy"'), False, map_location=[52.5195,13.3921], zoom=15)
m

no coords for edge (941, 844) wayid: [1195034182]
no coords for edge (844, 843) wayid: [589941657]


In [27]:
img_data = m._to_png(10)
Image.open(io.BytesIO(img_data)).save('Berlin-Mitte-FI.png')

## Treptow-Köpenick

### Harmonic Resistance

In [28]:
m = visfunctions.map_results(results.query('instance == "Treptow-Köpenick-Berlin-Germany" and experiment == "harmonicResistanceGreedy"'), False, color="#00FF00", map_location=[52.5195,13.3921], zoom=15)
m

In [29]:
img_data = m._to_png(10)
Image.open(io.BytesIO(img_data)).save('Berlin-TK-THR.png')

In [52]:
m = visfunctions.map_results(results.query('instance == "Treptow-Köpenick-Berlin-Germany" and experiment == "harmonicResistanceGreedy"'), False, color="#00FF00", map_location=[52.418,13.6132], zoom=12)
visfunctions.gjTKP.add_to(m)
m

In [53]:
img_data = m._to_png()
Image.open(io.BytesIO(img_data)).save('Berlin-TK-Boundary.png')

### Forest Index

In [56]:
m = visfunctions.map_results(results.query('instance == "Treptow-Köpenick-Berlin-Germany" and experiment == "forestIndexGreedy"'), False, map_location=[52.418,13.6132], zoom=12)
m

In [57]:
img_data = m._to_png(10)
Image.open(io.BytesIO(img_data)).save('Berlin-TK-FI.png')